In [1]:
import pandas as pd

In [ ]:

#df_item_user_group.to_json("DataSets/df_item_user_group.json", orient="records", lines=True)


In [64]:
def userdata( user_id : str ): 
    '''Esta funcion devuelve la cantidad de dinero gastado por el usuario, 
    el porcentaje de recomendación en base a reviews.recommend 
    y cantidad de items'''

    try:
        df_1 = pd.read_json("DataSets/df_user_price.json", lines=True)
        dinero_gastado = df_1['price'][df_1['user_id']==user_id]

        df_2 = pd.read_json("DataSets/df_porc_recomm.json", lines=True)
        recomendacion = df_2['Recomendacion'][df_2['user_id']==user_id]

        df_3 = pd.read_json("DataSets/df_item_user_group.json", lines=True)
        cantidad_item_id = df_3['Cantidad_item_id'][df_3['user_id']==user_id]

        
        # Verifico si el genero existe
        if dinero_gastado.empty or recomendacion.empty or cantidad_item_id.empty:
            return {"error": f"No se encontraron resultados para el usuario '{user_id}'."}

        resultado = {
                     "Usuario": user_id,
                     "Dinero gastado": dinero_gastado.iloc[0],
                     "Porcentaje de recomendacion": recomendacion.iloc[0],
                     "Cantidad de items": cantidad_item_id.iloc[0]
                    }

        return resultado

    except Exception as e:
        return {"error": str(e)}

In [67]:
user_id = '--000--'
userdata(user_id)

{'Usuario': '--000--',
 'Dinero gastado': 402.77,
 'Porcentaje de recomendacion': 100.0,
 'Cantidad de items': 58}

In [44]:
def funcion_countreviews(fecha_inicio_str, fecha_fin_str): # 'YYYY-MM-DD' y 'YYYY-MM-DD' : str
    '''Esta funcion devuelve la cantidad de usuarios que realizaron reviews entre las fechas dadas y,
    el porcentaje de recomendación de los mismos en base a reviews.recommend'''

    try:
        df = pd.read_json("DataSets/df_countreviews.json", lines=True)

        # Verifica el formato 'aaaa-mm-dd'
        if not fecha_inicio_str.count('-') == fecha_fin_str.count('-') == 2:
            return {"error": "El formato de fecha es incorrecto. Debe ser YYYY-MM-DD."}
        
        # Verifico si las fechan existen
        if df[df['posted_ok'] == fecha_inicio_str].empty:
            return {"error": f"La fecha '{fecha_inicio_str}' de inicio no existe."}
        
        if df[df['posted_ok'] == fecha_fin_str].empty:
            return {"error": f"La fecha '{fecha_fin_str}' de fin no existe."}

        condicion = (df['posted_ok'] >= fecha_inicio_str) & (df['posted_ok'] <= fecha_fin_str)
        recomendacion = df[condicion][df[condicion]['recommend']==True].shape[0]
        porc_recomendacion = recomendacion / df[condicion].shape[0] * 100

        resultado = {
                     "Cantidad de usuarios": df[condicion].shape[0],
                     "Porcentaje de recomendación": round(porc_recomendacion, 2)
                    }
        
        return resultado

    except Exception as e:
        return {"error": str(e)}


In [45]:
fecha_inicio_str = '2011-12-27'
fecha_fin_str = '2011-12-27'

funcion_countreviews(fecha_inicio_str, fecha_fin_str)

{'Cantidad de usuarios': 60, 'Porcentaje de recomendación': 96.67}

In [13]:
def genre(genero): 
    '''Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos 
    analizado bajo la columna PlayTimeForever.'''
    try:
        df = pd.read_json("DataSets/df_genre.json", lines=True)
        genero = genero.lower()  # Convertir a minúsculas

        # Verifico si el género existe
        if df[df['genres'] == genero].empty:
            return {"error": f"No se encontraron resultados para el género '{genero}'."}

        result = df['ranking'][df['genres'] == genero].iloc[0]
        return {"Genero": genero, "Ranking": result}
    except Exception as e:
        return {"error": str(e)}


In [14]:
genre('ActIon')

{'Genero': 'action', 'Ranking': 1}

In [39]:
def userforgenre(genero): 
    
    '''Esta funcion devuelve el top 5 de usuarios con más horas de juego 
    en el género dado, con su URL (del user) y user_id'''

    try:
        df = pd.read_json("DataSets/df_userforgenre.json", lines=True)
        genero = genero.lower()  # Convierto a minúsculas
        ranking = df[['ranking']][df['genres']==genero]
        user = df[['user_id']][df['genres']==genero]
        url = df[['user_url']][df['genres']==genero]
        
        # Verifico si el genero existe
        if ranking.empty or user.empty or url.empty:
            return {"error": f"No se encontraron resultados para el género '{genero}'."}
        
        # Creao un diccionario con los resultados
        resultados = {
            "Ranking": ranking.values.tolist(),
            "User_ID": user.values.tolist(),
            "User_URL": url.values.tolist()
                    }
        return resultados
    
    except Exception as e:
        return {"error": str(e)}

In [41]:
userforgenre('Sports')

{'Ranking': [[1], [2], [3], [4], [5]],
 'User_ID': [['76561197995725436'],
  ['76561198065902279'],
  ['express157'],
  ['76561198045342815'],
  ['1silentsniper1']],
 'User_URL': [['http://steamcommunity.com/profiles/76561197995725436'],
  ['http://steamcommunity.com/profiles/76561198065902279'],
  ['http://steamcommunity.com/id/express157'],
  ['http://steamcommunity.com/profiles/76561198045342815'],
  ['http://steamcommunity.com/id/1silentsniper1']]}

In [2]:
def developer(desarrollador): 
    '''Esta funcion devuelve la cantidad de items y porcentaje de contenido Free por año 
    según empresa desarrolladora.'''
    try:
        df = pd.read_json("DataSets/df_developer.json", lines=True)
        desarrollador = desarrollador.lower()  # Convierto a minúsculas
        
        # Filtro el DF por desarrollador y verifico si el desarrollador existe
        df_filtrado = df[df['developer'] == desarrollador]

        if df_filtrado.empty:
            return {"error": f"No se encontraron resultados para el desarrollador '{desarrollador}'."}
        
        # Agrupo por año y cuento los registros
        registros = df_filtrado.groupby('año')['item_id'].count().reset_index(name='cantidad_item')
        
        # Cuento los registros donde el precio = cero
        free = df_filtrado[df_filtrado['price'] == 0.0].groupby('año')['price'].count().reset_index(name='contenido_free')
        
        # Combino ambos DataFrames
        resultados = registros.merge(free, on='año', how='left')
        
        # Calculo el porcentaje de contenido gratuito
        resultados['porcentaje_free'] = (resultados['contenido_free'] / resultados['cantidad_item']) * 100
        
        # Si no hay valores price, le ongo cero
        resultados['porcentaje_free'].fillna(0, inplace=True)
        
        # Diccionario de resultados
        resultados_dict = {
            "Año": resultados['año'].tolist(),
            "Cantidad de Items": resultados['cantidad_item'].tolist(),
            "Porcentaje de contenido Free": resultados['porcentaje_free'].tolist()
        }
        return resultados_dict
    
    except Exception as e:
        return {"error": str(e)}


In [6]:
developer('poppErmost productions')

{'Año': [2013, 2018],
 'Cantidad de Items': [1, 5],
 'Porcentaje de contenido Free': [100.0, 0.0]}

In [19]:
def sentiment_analysis(año_str: str): 
    '''Esta funcion devuelve una lista con la cantidad de registros de reseñas de usuarios 
    que se encuentren categorizados con un análisis de sentimiento según el año de lanzamiento.
    '''
    try:
        # Convierto el año a entero
        año = int(año_str)
        
        df = pd.read_json("DataSets/df_sentiment.json", lines=True)
        
        # Verifico que el año este en el DataFrame
        if año not in df['ano'].unique():
            return {"error": f"No se encontraron registros para el año {año}"}

        negativos = int(df['cantidad_0'][df['ano']==año].iloc[0])
        neutros = int(df['cantidad_1'][df['ano']==año].iloc[0])
        positivos = int(df['cantidad_2'][df['ano']==año].iloc[0])

        return {"Negativos": negativos, "Neutros": neutros, "Positivos" : positivos}
    
    except Exception as e:
        return {"error": str(e)}


In [25]:
sentiment_analysis(1998)

{'Negativos': 2, 'Neutros': 66, 'Positivos': 14}